In [2]:
!pip install cv2
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install numpy

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Omen\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Omen\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmotionRecognitionCNN(nn.Module):
    def __init__(self, num_classes):
        super(EmotionRecognitionCNN, self).__init__()

        self.features = nn.Sequential(
            # Первый блок сверток
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # 3 канала для цветных изображений
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Уменьшение размерности

            # Второй блок сверток
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Уменьшение размерности

            # Третий блок сверток
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Уменьшение размерности
        )

        # Классификатор
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 1024),  # Входной размер: 256 каналов после сверток
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)  # Выходной слой с числом классов
        )

    def forward(self, x):
        x = self.features(x)  # Применение сверток
        x = x.view(x.size(0), -1)  # Разворачиваем тензор для подачи в fully connected слои
        x = self.classifier(x)  # Применение классификатора
        return x




In [6]:
import cv2
import torch
import torch.nn.functional as F
import numpy as np
from torchvision import transforms

# Загрузка обученной модели
model = EmotionRecognitionCNN(num_classes=6)
model.load_state_dict(torch.load('emotion_recognition_model_V4.pth'))
model.eval()

# Загрузка классификатора Хаара для обнаружения лиц
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Преобразования для предобработки изображения
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Используем три канала
])

# Метки эмоций
emotions = ['Angry', 'Fearful', 'Happy', 'Sad', 'Surprised', 'Neutral']

# Запуск камеры
cap = cv2.VideoCapture(0)

while True:
    # Захват изображения с камеры
    ret, frame = cap.read()
    if not ret:
        break

    # Обнаружение лиц на изображении
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Извлечение лица
        face = frame[y:y + h, x:x + w]

        # Преобразование для модели
        face_img = transform(face).unsqueeze(0)  # Добавляем размер батча

        # Предсказание эмоции
        with torch.no_grad():
            output = model(face_img)
            emotion_prediction = F.softmax(output, dim=1)
            emotion_label = torch.argmax(emotion_prediction).item()

        # Метка предсказанной эмоции
        emotion_text = emotions[emotion_label]

        # Отображение предсказанной эмоции на изображении
        cv2.putText(frame, emotion_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Показ изображения
    cv2.imshow('Emotion Recognition', frame)

    # Прерывание по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение камеры и закрытие окон
cap.release()
cv2.destroyAllWindows()



C:\Users\Omen\AppData\Local\Temp\ipykernel_18336\893831113.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('emotion_recognition_model_V4